In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

import qubic
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Calibration.Qselfcal import scatter_plot_FP, get_TEScoordinates_ONAFP
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing

In [ ]:
nside = 128

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
def get_dict(comm, key='in'):
    """QUBIC dictionary.

    Method to modify the qubic dictionary.

    Parameters
    ----------
    key : str, optional
        Can be "in" or "out".
        It is used to build respectively the instances to generate the TODs or to reconstruct the sky maps,
        by default "in".

    Returns
    -------
    dict_qubic: dict
        Modified QUBIC dictionary.

    """

    args = {
        "npointings": 30,
        "nf_recon": 2,
        "nf_sub": 20,
        "nside": nside,
        "MultiBand": True,
        "period": 1,
        "RA_center": 0,
        "DEC_center": -57,
        "filter_nu": 150 * 1e9,
        "noiseless": False,
        "comm": comm,
        "dtheta": 15,
        "nprocs_sampling": 1,
        "nprocs_instrument": comm.Get_size(),
        "photon_noise": True,
        "nhwp_angles": 3,
        #'effective_duration':3,
        "effective_duration150": 3,
        "effective_duration220": 3,
        "filter_relative_bandwidth": 0.25,
        "type_instrument": "two",
        "TemperatureAtmosphere150": None,
        "TemperatureAtmosphere220": None,
        "EmissivityAtmosphere150": None,
        "EmissivityAtmosphere220": None,
        "detector_nep": float(4.7e-17),
        "synthbeam_kmax": 2,
        "synthbeam_fraction": 1,
        "beam_shape": 'gaussian',
        "debug":False,
    }

    ### Get the default dictionary
    dictfilename = "dicts/pipeline_demo.dict"
    dict_qubic = qubicDict()
    dict_qubic.read_from_file(dictfilename)

    for i in args.keys():

        dict_qubic[str(i)] = args[i]

    return dict_qubic

In [ ]:
# Build MPI communicator and qubic dict
comm = MPI.COMM_WORLD
dict_qubic = get_dict(comm)

In [ ]:
# Build scanning strategy
sampling = get_pointing(dict_qubic)

# Initialize QubicInstrument instance
q = QubicInstrument(dict_qubic)

In [ ]:
# Number of sub-acquisitions
nsub = 15

In [ ]:
# Initialize the Qubic multi-frequency acquisition
# I multiply by 2 Nsub to have Nsub bands in each band, the rest of the code will only use the first Nsub bands (meaning the ones in the 150 GHz band)
acq = QubicMultiAcquisitions(dict_qubic, nsub*2, 2, sampling=sampling)

In [ ]:
# Build a list that contains all sub-ProjectionOperators, meaning the ProjectionOperators for each sub-bands
Proj_list = []
for i in range(nsub):
    Proj_list.append(acq.subacqs[i].get_projection_operator())

In [ ]:
# Create uniform maps of "1"
input_maps = np.ones((nsub, hp.nside2npix(nside), 3))

# Build the list of sub-TODs
tod = []
for i in range(nsub):
    tod.append(Proj_list[i](input_maps[i]))

In [ ]:
print(tod[0].shape)

In [ ]:
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 0] - tod[i][:, 0, 0])/tod[i][:, 0, 0])
    # Need to remove 'nan' due to division by 0
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.legend()
plt.title(r"$H = \mathcal{P}_{roj}$ - Intensity map")
plt.ylabel(r'$\frac{| H(Imap)^{i+1} - H(Imap)^i |}{| H(Imap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][..., 1] - tod[i][..., 1])/tod[i][..., 1])
    # Need to remove 'nan' due to division by 0
    mean_diff = np.mean(diff[~np.isnan(diff)])
    plt.plot(acq.allnus[i], mean_diff, '.')
plt.legend()
plt.title(r"$H = \mathcal{P}_{roj}$ - Q map")
plt.ylabel(r'$\frac{| H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

Plot the difference between two successive tods, at a given time, on the focal plane. The polarisation case seems very strange with respect to the intensity case.

In [ ]:
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = np.abs(tod[10]-tod[9])/tod[9]*100
scatter_plot_FP(q, xTES, yTES, np.mean(relative_diff[...,0], axis=1), frame='ONAFP', s=150, title=r'Intensity map - $\frac{| \mathcal{P}_{roj}(Imap)^{20} - \mathcal{P}_{roj}(Imap)^{19} |}{| \mathcal{P}_{roj}(Imap)^{19} |} x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = np.abs(tod[10]-tod[9])/tod[9]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $\frac{| \mathcal{P}_{roj}(Qmap)^{20} - \mathcal{P}_{roj}(Qmap)^{19} |}{| \mathcal{P}_{roj}(Qmap)^{19} |} x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
P = Proj_list[-1]

alpha_from_r22 = 0.5 * np.arccos(P.matrix.data.r22 / P.matrix.data.r11)
alpha_from_r32 = 0.5 * np.arcsin(- P.matrix.data.r32 / P.matrix.data.r11)

cos2_sin2 = (P.matrix.data.r22**2 + P.matrix.data.r32**2) / P.matrix.data.r11**2

In [ ]:
plt.plot(alpha_from_r22[0], label=r'$\alpha_{22}$')
plt.plot(alpha_from_r32[0], label=r'$\alpha_{32}$')

plt.legend()
plt.xlabel('Peaks index')
plt.ylabel('Angle (rad)')

In [ ]:
plt.plot(alpha_from_r22[0], label=r'$\alpha_{22}$')
plt.plot(alpha_from_r32[0], label=r'$\alpha_{32}$')
plt.plot(np.abs(alpha_from_r32[0]), label=r'$|\alpha_{32}|$')

plt.legend()
plt.xlabel('Peaks index')
plt.ylabel('Angle (rad)')

In [ ]:
plt.hlines(0, 0, len(alpha_from_r22[0]), 'grey', linestyle='--', alpha=0.5)
plt.plot(alpha_from_r22[0] - (alpha_from_r32[0]), label=r'$\alpha_{22} - \alpha_{32}$')
plt.plot(alpha_from_r22[0] - np.abs(alpha_from_r32[0]), label=r'$\alpha_{22} - |\alpha_{32}|$')
plt.plot(alpha_from_r22[0] - (alpha_from_r32[0] % (np.pi/2)), label=r'$\alpha_{22} - \alpha_{32} \ mod \ (\pi/2)$')
plt.legend()
plt.xlabel('Peaks index')
plt.ylabel('Angle (rad)')